# Projeto 1 - Ciência dos Dados

Nome: Alison Araujo

Nome: Gabrielly Carneiro

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
from unidecode import unidecode
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tqdm import tqdm
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('word_tokenize')
stopwordsdic = stopwords.words('portuguese')

from spacy import load
nlp = load('pt_core_news_sm')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aliso\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aliso\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Error loading word_tokenize: Package 'word_tokenize' not
[nltk_data]     found in index


In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\aliso\OneDrive - Insper - Institudo de Ensino e Pesquisa\Insper\2023.2\Ciência de Dados\C.DadosP1


Carregando a base de dados com os tweets classificados manualmente:

In [3]:
train = pd.read_excel('dados_treino.xlsx')
train.head(5)

,Mensagem,Acionável/Direcionável/Não Acionável
0,"Para quem gosta de poemas simples, esse é o li...",N
1,Quando vi o lançamento pensei que finalmente p...,D
2,"É incrível como esses escritores e ""intelectua...",N
3,"Se desse pra devolver eu devolvia, nao é por e...",N
4,"Futilidade, inutilidade, desperdício de papel,...",N


In [4]:
test = pd.read_excel('dados_teste.xlsx')
test.head(5)

,Mensagem,Acionável/Direcionável/Não Acionável
0,"O livro é prolixo, redundante, doentio. Sou su...",N
1,"Bom livro e história envolvente. Porém, o leit...",D
2,"Fala tudo que todos já sabem, sem falar que nã...",N
3,Ganhei na compra do Kindle. Não é estilo de li...,N
4,"Custa crer que um livro tão medíocre, embora m...",N


___
## Classificador automático


Na primeira etapa, de classificação manual, consideramos três targets para os reviews: Acionável, Direcionável e Não Acionável. 

- Acionável: para ser considerado "acionável" ("A") o review deve ser passível de alguma ação pela Amazon, ou seja, o review deve ser sobre entrega, estado do produto, contato com o suporte, etc. 
- Direcionável: para o target "direcionável" ("D") foram considerados comentários relativos à editora, como qualidade do material do livro, preço do livro e do e-book, tradução e edição. 
- Não Acionáveis: por fim, os não acionáveis ("NA") eram comentários relativos ao autor, ao apreço pelo conteúdo do livro, ou comentários irrelavantes.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

Funções úteis:

In [5]:
#Criar uma função que transforma as frases da planilha em um texto só 
    #(Será útil para criar o dicionário com as palavras)

def transforma_em_string(coluna):
    texto = ''
    for linha in coluna:
        texto += linha + ' '
    return texto    

In [6]:
#Criar uma função que limpa todas as pontuações
#recebe um texto
#vou utilizar ela na função limpa tudo
def cleanup(text):
    punctuation = r'[´"\'!-.:?;$,/~^_=+*&¨%$#@|\{}()[\]]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

In [7]:
#Cria uma função que limpa os espaços duplicados
#vou utilizar ela na função limpa tudo
def limpa_espaco(text):
    punctuation = r'[\n]'  # Adicione os caracteres desejados aqui
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

In [8]:
#Criando uma função para remover emoji
#vou utilizar ela na função limpa tudo
def remove_emoji(text):
    text_without_emojis = unidecode(text)
    return text_without_emojis

In [9]:
#Criando a função de stopwords
#vou utilizar ela na função limpa tudo
#vai receber o texto limpo pelas outras funções
def stopwords(texto):
    palavras = word_tokenize(texto, language='portuguese') # Tokenize é analisar palavras individualmente, basicamente
    palavras_sem_stopword = []
    for palavra in palavras:
        if palavra not in stopwordsdic:
            palavras_sem_stopword.append(palavra)
    # Reúna as palavras sem stopwords em uma string novamente
    texto_sem_stopword = ' '.join(palavras_sem_stopword)
    return texto_sem_stopword

In [10]:
#Criando a função de lematização
#vou utilizar ela na função limpa tudo
#vai receber o texto limpo pelas outras funções, incluindo limpeza de stopwords

def lemmat(texto):
    doc = nlp(texto)
    lemmat_radicais = []
    for radicais in doc:
        lemmat_radicais.append(radicais.lemma_)
    texto_lemmat = ' '.join(lemmat_radicais)    
    return texto_lemmat

In [11]:
# Cria uma função que reúna as funções de limpeza
def limpa_tudo(mensagem):
    #Aplicando a função de limpeza de pontuação
    texto = cleanup(mensagem)
    #Deixando tudo em letra minúscula
    texto = texto.lower()
    #Removendo emoji
    texto = remove_emoji(texto)
    #Aplicando a função de limpeza de espaço
    texto = limpa_espaco(texto)
    #Removendo stopwords
    texto = stopwords(texto)
    #Realiza lemmatização
    texto = lemmat(texto)          # removi pois demora muito para classificar 
    return texto

In [12]:
#Cria uma função que limpa as linhas da planilha e adiciona uma coluna com as mensagens limpas à planilha
def mensagem_limpa(planilha):   #recebe a planilha e cria uma nova planilha com a coluna de mensagem limpa com as frases limpas
    planilha_limpa = planilha.copy()
    planilha_limpa['Mensagem Limpa'] = [limpa_tudo(x) for x in list(planilha['Mensagem'])]
    return planilha_limpa    #retorna a planilha modificada

In [13]:
#Cria função que cria o vocabulário de tudo
def cria_vocabulario(coluna_mensagem_limpa_da_planilha_limpa):                       #recebe uma coluna da planilha
    lista_palavras = transforma_em_string(coluna_mensagem_limpa_da_planilha_limpa)
    lista_palavras = lista_palavras.split()
    return lista_palavras     #devolve uma lista com as palavras separadas

In [14]:
#Cria função que retorna uma lista sem as palavras repetidas
def remove_repeticao(lista_de_palavras):
    dic = set(lista_de_palavras)
    lista_vocabulario_sem_repeticao = list(dic)
    return lista_vocabulario_sem_repeticao

In [15]:
# Cria uma função que guarda as palavras em pd.Series e calculas frequencia (relativa e absoluta) das palavras
#recebe uma lista (devolvida pelo cria vocabulário, preferencialmente)
def cria_pdseries(lista):
    tabela = pd.Series(lista)
    return tabela

#Cria uma função que retorna a frequência absoluta de cada palavra no texto
#recebe uma tabela de pd
def freq_abs(tabela):
    absoluta = tabela.value_counts()
    return absoluta

#Cria uma função que retorna a frequência relativa de cada palavra no texto
def freq_rel(tabela):
    relativa = tabela.value_counts(True)
    return relativa

In [16]:
#Cria função de divisão com base nas categorias/targets
## Cria uma função que recebe a planilha e o target e 
# retorna o total de palavras e as tabelas de frequencia absoluta e relativa e o vocabulario sem palavras repetidas, nessa ordem
#vou falar que ela recebe a planilha limpa já

def divisao_categorias(planilha_limpa, target):
    #Etapa de divisão de categorias     
    #criou uma nova planilha apenas com as linha com aquele target                
    filtro_target = planilha_limpa.loc[planilha_limpa['Acionável/Direcionável/Não Acionável'] == target]
    vocab_target = cria_vocabulario(filtro_target["Mensagem Limpa"]) #vocabulario daquele target
    df_vocab_target = cria_pdseries(vocab_target)
    freq_rel_target = freq_rel(df_vocab_target)
    freq_abs_target = freq_abs(df_vocab_target)   #frequencia de palavras daquele target
    total_target = freq_abs_target.sum()  #total de palavras daquele target
    #criar vocabulario limpo
    vocab_target_sr = remove_repeticao(vocab_target)   #vocabulario do target sem palavras repetidas
    return total_target, freq_abs_target, freq_rel_target, vocab_target_sr

In [17]:
# Função que realiza a divisão de toda a planilha 
# # Função que recebe a planilha já limpa e retorna:
# o total de palavras com repetição [0] (int)
# a tabela com a frequência absoluta de cada palavra na planilha [1] (Series)
# a tabela com a frequência relativa de cada palavra na planilha [2] (Series)
# a lista com todas as palavras da planilha sem repetição [3] (list)
def divisao_planilhas(planilha_limpa):
    
    coluna_limpa = planilha_limpa["Mensagem Limpa"]                   #separa só a coluna "Mensagem Limpa"
    vocab_planilha = cria_vocabulario(coluna_limpa)             #lista de palavras na coluna "Mensagem Limpa"
    df_vocab_planilha = cria_pdseries(vocab_planilha)       #coloca essa lista em um df
    freq_abs_planilha = freq_abs(df_vocab_planilha)         #calcula a frequência absoluta de cada palavra
    freq_rel_planilha = freq_rel(df_vocab_planilha)         #calcula a frequência relativa de cada palavra
    total_planilha = freq_abs_planilha.sum()                #calcula o total de palavras com repetição
    vocab_planilha_sr = remove_repeticao(vocab_planilha)       #vocabulario sem repeticao (é uma lista)

    return total_planilha, freq_abs_planilha, freq_rel_planilha, vocab_planilha_sr


In [18]:
# Função que cria um dicionário com as palavras e as frequencias absolutas

#vou criar um dicionário com todas as palavras e seus valores de 
# frequencia absoluta para cada target
# aí vai ficar mais rapido de encontrar os valores
# do que ter que suavizar pra cada palavra toda vez
# que chamar o loop
def dicionario_prob_palavra_dado_target_treino(planilha_treino_limpa, target):
    coluna_limpa = planilha_treino_limpa["Mensagem Limpa"]
    dados_target = divisao_categorias(planilha_treino_limpa, target)
    # frequencias_relativas = dados_target[2]
    frequencias_absolutas = dados_target[1]

    dic_treino = {} 
    
    for frase in coluna_limpa:
        frase = frase.split()
        for palavra in frase:
            if palavra not in dic_treino:
                if palavra in frequencias_absolutas:
                    freq_abs_palavra = frequencias_absolutas[palavra]
                    dic_treino[palavra] = freq_abs_palavra
    return dic_treino

In [19]:
# Função que calcula probabidade da frase dado target realizando suavização de LaPlace
# Probabilidade da frase dado target mais suavização de LaPlace
def frase_dado_target(frase_do_teste, target, planilha_treino_limpa):
    prob_frase_dado_classe = 1
    frase_do_teste =  frase_do_teste.split()
    dic_probabilidades_por_classe_treino = dicionario_prob_palavra_dado_target_treino(planilha_treino_limpa, target)   #dicionario com as palavras e suas frequencias absolutas
    dados_target = divisao_categorias(planilha_treino_limpa, target)
    qtdd_palavras = dados_target[0]                  #quantidade de palavras no target informado
    qtdd_palavras_sem_repeticao = len(dados_target[3]) 
    
    alfa = 0.01
    for palavra in frase_do_teste:
        if palavra in dic_probabilidades_por_classe_treino:
            freq_abs_palavra = dic_probabilidades_por_classe_treino[palavra]
            #APLIQUEI A SUAVIZAÇÃO DE LA PLACE
            prob_palavra_dado_classe = (freq_abs_palavra + alfa)/(qtdd_palavras + alfa*qtdd_palavras_sem_repeticao)
            prob_frase_dado_classe *= prob_palavra_dado_classe
        else:
            freq_abs_palavra = 0
            #APLIQUEI A SUAVIZAÇÃO DE LA PLACE
            prob_palavra_dado_classe = (freq_abs_palavra + alfa)/(qtdd_palavras + alfa*qtdd_palavras_sem_repeticao)
            prob_frase_dado_classe *= prob_palavra_dado_classe
    return prob_frase_dado_classe

In [20]:
#Função de classificação das frases (quanto a acionável, não acionável e direcionável)
#recebe a função já limpa
def classificador(frase_do_teste, planilha_treino):
    
    #Extraindo as informações
    total_planilha_N = divisao_categorias(planilha_treino, "N")[0]
    total_planilha_D = divisao_categorias(planilha_treino, "D")[0]
    total_planilha_A = divisao_categorias(planilha_treino, "A")[0]
    total_planilha = divisao_planilhas(planilha_treino)[0]
    
    #Cálculo das probabilidades
    P_frase_dado_A = frase_dado_target(frase_do_teste, "A", planilha_treino)
    P_frase_dado_D = frase_dado_target(frase_do_teste, "D", planilha_treino)
    P_frase_dado_N = frase_dado_target(frase_do_teste, "N", planilha_treino)
    P_N = total_planilha_N/total_planilha         #probabilidade de estar na categoria N
    P_D = total_planilha_D/total_planilha         #probabilidade de estar na categoria D
    P_A = total_planilha_A/total_planilha         #probabilidade de estar na categoria A
    
    P_A_dado_frase = P_frase_dado_A*P_A
    P_N_dado_frase = P_frase_dado_N*P_N
    P_D_dado_frase = P_frase_dado_D*P_D
        
    #Classificação
    if P_A_dado_frase >= P_D_dado_frase and P_A_dado_frase >= P_N_dado_frase:    #na classificação manual quando impatava a prioridade era acionável
        return "A"
    elif P_D_dado_frase > P_N_dado_frase and P_D_dado_frase > P_A_dado_frase:
        return "D"
    elif P_N_dado_frase > P_D_dado_frase and P_N_dado_frase > P_A_dado_frase:
        return "N"
    else:
        return "Houve um impasse"    
    

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [21]:
# Função que classifica todas as mensagens da planilha de teste com base na planilha de treino
#Fazer meu classificador classificar todas as frases da planilha de teste
#recebe a planilha inteira, não precisa realizar nenhuma limpeza
def classifica_planilha(planilha_teste, planilha_treino):
    planilha_teste_limpa = mensagem_limpa(planilha_teste)
    planilha_treino_limpa = mensagem_limpa(planilha_treino)
    avaliacoes_limpas = planilha_teste_limpa["Mensagem Limpa"]
    #fez uma cópia da planilha para nao alterar a antiga
    planilha_nova = planilha_teste_limpa.copy()
    # Criar uma lista para armazenar os resultados do classificador
    resultados = []
    for frase in tqdm(avaliacoes_limpas, desc="Classifying", unit="rows"):
        resultado = classificador(frase, planilha_treino_limpa)
        resultados.append(resultado)
    # Atribuir a lista de resultados à coluna 'Classificador Automático'
    planilha_nova['Classificador Automático'] = resultados
    return planilha_nova

classifica_planilha(test, train).head()

Classifying: 100%|██████████| 250/250 [00:13<00:00, 18.40rows/s]


,Mensagem,Acionável/Direcionável/Não Acionável,Mensagem Limpa,Classificador Automático
0,"O livro é prolixo, redundante, doentio. Sou su...",N,livro prolixo redundante doentio suspeito fala...,N
1,"Bom livro e história envolvente. Porém, o leit...",D,bom livro histor envolvente por leitor tera co...,D
2,"Fala tudo que todos já sabem, sem falar que nã...",N,falar tudo todo ja saber falar nao embasamento...,N
3,Ganhei na compra do Kindle. Não é estilo de li...,N,ganhar compra Kindle nao estilo livro gostar,N
4,"Custa crer que um livro tão medíocre, embora m...",N,custar crer livro taoer mediocre embora avalia...,N


In [22]:
#Função que faz as comparações de acurácia e classificação das mensagens
def divide_comparativos(planilha_treino, planilha_teste):

    planilha = classifica_planilha(planilha_teste, planilha_treino)
    comparador = pd.crosstab(planilha['Classificador Automático'], planilha['Acionável/Direcionável/Não Acionável'],normalize=True, margins=True)

    acuracia = comparador.iloc[0]['A'] + comparador.iloc[1]['D'] + comparador.iloc[2]['N'] 

    pct_falsos_A = comparador.iloc[0]['D'] + comparador.iloc[0]['N']
    pct_falsos_D = comparador.iloc[1]['A'] + comparador.iloc[1]['N']
    pct_falsos_N = comparador.iloc[2]['D'] + comparador.iloc[2]['A']
    pct_verdadeiros_A = comparador.iloc[0]['A']
    pct_verdadeiros_D = comparador.iloc[1]['D']
    pct_verdadeiros_N = comparador.iloc[2]['N'] 

    return acuracia*100, pct_falsos_A*100, pct_falsos_D*100, pct_falsos_N*100, pct_verdadeiros_A*100, pct_verdadeiros_D*100, pct_verdadeiros_N*100, comparador

resultados = divide_comparativos(train,test)

verdadeiros_positivos = resultados[0]
falsos_acionaveis = resultados[1]
falsos_direcionaveis = resultados[2]
falsos_nao_acionaveis = resultados[3]
verdadeiros_acionaveis = resultados[4]
verdadeiros_direcionaveis = resultados[5]
verdadeiros_naoacionaveis = resultados[6]
df_crosstab = resultados[7]

Classifying: 100%|██████████| 250/250 [00:14<00:00, 17.24rows/s]


In [23]:
#printando resultados
print("A acurácia foi {0:.2f}% (mensagens corretamente classificadas)".format(verdadeiros_positivos))
print("A procentagem de falsos acionáveis foi {0:.2f}% (mensagens incorretamente classificadas como acionáveis)".format(falsos_acionaveis))
print("A procentagem de falsos direcionáveis foi {0:.2f}% (mensagens incorretamente classificadas como direcionáveis)".format(falsos_direcionaveis))
print("A procentagem de falsos não acionáveis foi {0:.2f}% (mensagens incorretamente classificadas como não acionáveis)".format(falsos_nao_acionaveis))
print("A procentagem de verdadeiros acionáveis foi {0:.2f}% (mensagens corretamente classificadas como acionáveis)".format(verdadeiros_acionaveis))
print("A procentagem de verdadeiros direcionáveis foi {0:.2f}% (mensagens corretamente classificadas como direcionáveis)".format(verdadeiros_direcionaveis))
print("A procentagem de verdadeiros não acionáveis foi {0:.2f}% (mensagens corretamente classificadas como não acionáveis)".format(verdadeiros_naoacionaveis))
#cruzamento de dados da planilha
df_crosstab

A acurácia foi 83.60% (mensagens corretamente classificadas)
A procentagem de falsos acionáveis foi 6.80% (mensagens incorretamente classificadas como acionáveis)
A procentagem de falsos direcionáveis foi 6.40% (mensagens incorretamente classificadas como direcionáveis)
A procentagem de falsos não acionáveis foi 3.20% (mensagens incorretamente classificadas como não acionáveis)
A procentagem de verdadeiros acionáveis foi 10.80% (mensagens corretamente classificadas como acionáveis)
A procentagem de verdadeiros direcionáveis foi 13.20% (mensagens corretamente classificadas como direcionáveis)
A procentagem de verdadeiros não acionáveis foi 59.60% (mensagens corretamente classificadas como não acionáveis)


Acionável/Direcionável/Não Acionável,A,D,N,All
Classificador Automático,,,,
A,0.108,0.028,0.040,0.176
D,0.036,0.132,0.028,0.196
N,0.000,0.032,0.596,0.628
All,0.144,0.192,0.664,1.000


___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas categorias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por categoria (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**

Dica: apresentar um grafico com testes mostrando quais limpezas melhoraram a acurácia do nosso classificador (lemmatization, stopwords, etc)

REtirar algumas palavras para ver se melhora a qualidade do classificador (exemplo palavra não)